In [0]:
########################
#Install Dependancies
########################
#Install openpyxl so can export as excel (only required to once for your computer)
# %pip install openpyxl
# %pip install pycountry
# dbutils.library.restartPython()

In [0]:
####INSTRUCTIONS FOR CLONING FOR NEW STATE

#1) Change in Cell 3 - Change this_notebook_state to value for State - this should be the name of the folder wheer the gold data is stored
#2) Change in Cell 3 - Change The import Statement >>> import Test_Functions.AppealSubmitted_Tests as as_tests >> to the function
#  file containing the test code for this new state and a new name for the import to be used
#3) Change in Cell 3 - Change child_fields_to_exclude, if you need to override the testing for a field tested in an earlier state
#4) Change in Cell 5 - The Child Notebook you wish to Call.. Usualy previous as they are based on last one
#5) Change in Cell 6 - Change Import line >>> importlib.reload(as_tests) to the name you give in step 2
#6) Start Creating Tests in cell 7 (demo tests) , alwaysd inserting new cells for each field group but ensuring that they are always before the last cell (currently cell 8) which produces the test report / return data back to parent notebook


In [0]:
########################
#Setup for Tranformation Tests
########################

#####################
#Define This notebook name
this_notebook_state = "appealSubmitted"

#Setup Test Reporting
from datetime import datetime, timedelta
run_user = spark.sql("SELECT current_user()").first()[0]
run_tag = "Testing Transformation Tests"
run_by_automation_name = "Transformation_Tests"
#Capture Test Start datetime
run_start_datetime = datetime.now()
#####################

#Setup Notebook Parameters (Defaulting to Payment Pending and Running all tests)
dbutils.widgets.text("state_under_test", "")

#fields_to_exclude : Should be a comma separated list of fields to exclude 
dbutils.widgets.text("fields_to_exclude", "")

# Read parameters
state_under_test = dbutils.widgets.get("state_under_test")
fields_to_exclude = dbutils.widgets.get("fields_to_exclude")

#Set Default Values if not called from another Notebook
if state_under_test == "" :    
    state_under_test = this_notebook_state

#Get Fields to Exclude
if fields_to_exclude != "":    
    fields_to_exclude = [item.strip() for item in fields_to_exclude.split(",")]
else:
    fields_to_exclude = []
    
#List of Fields to Exclude in Child Notebooks Called
child_fields_to_exclude = ["feeWithHearing"]
#Combine any Fields to Exclude
fields_to_exclude.extend(child_fields_to_exclude)

print(f"Fields to exclude = {str(fields_to_exclude)}")
print(f"Testing State = {state_under_test}")

#Restart Python When needed (when changed tests)
# dbutils.library.restartPython()

#Import Tests
import Test_Functions.AppealSubmitted_Tests as as_tests

#Import models.test_result as test_result
from models.test_result import TestResult
# TestResult.test_from_state = "appealSubmitted"

#Import asdict to convert Dataclass to Dictionary
from dataclasses import asdict
import os

#Setup Global Variables
all_test_results = []
current_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
base_path = current_path.rsplit("/", 1)[0] + "/"
#Below will be replaced eventually to store the reults in a spark table
test_results_path= "/Workspace/Users/" + run_user + "/Results/Transformation_Tests"
os.makedirs(test_results_path, exist_ok=True)


In [0]:
#Load Config and Setup Enviorment Variables
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()
 
# print(f"env_code: {lz_key}")  # This won't be redacted
# print(f"env_name: {env_name}")  # This won't be redacted
 
KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
# print(f"KeyVault_name: {KeyVault_name}")
 
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")
 
# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"
external_storage = f"ingest{lz_key}external{env_name}"
  
# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{external_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{external_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{external_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{external_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{external_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
  
# Setting variables for use in subsequent cells
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
audit_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/AUDIT/{state_under_test}"
gold_path = f"abfss://gold@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/{state_under_test}"
 
 
# Print all variables
# variables = {
#     # "read_hive": read_hive,
    
#     "bronze_path": bronze_path,
#     "silver_path": silver_path,
#     "audit_path": audit_path,
#     "gold_path": gold_path,
#     "key_vault": KeyVault_name,
#     "AppealState": state_under_test
 
# }
 
# display(variables)

import json

#Get Latest Json Folder
json_location = dbutils.fs.ls(f"{gold_path}/")[-1]
latest_json_location = json_location.name
dbutils.fs.ls(f"{gold_path}/{latest_json_location}")

#Set Paths
try: 
    json_path = f"{gold_path}/{latest_json_location}/JSON/"
    # json_path = f"{gold_path}/{latest_json_location}/INVALID_JSON/"
    M1_silver = f"{silver_path}/silver_appealcase_detail"
    M1_bronze = f"{bronze_path}/bronze_appealcase_crep_rep_floc_cspon_cfs"
    M2_silver = f"{silver_path}/silver_caseapplicant_detail"
    M3_silver = f"{silver_path}/silver_status_detail"
    C = f"{silver_path}/silver_appealcategory_detail"
    bhc = f"{bronze_path}/bronze_hearing_centres"    
except:
    print(f"Error during fetch: {str(e)}")

#Create and Load Dataframes
json_data = spark.read.format("json").load(json_path)
M1_silver = spark.read.format("delta").load(M1_silver)
M1_bronze = spark.read.format("delta").load(M1_bronze)
M2_silver = spark.read.format("delta").load(M2_silver)
M3_silver = spark.read.format("delta").load(M3_silver)
C = spark.read.format("delta").load(C)
bhc = spark.read.format("delta").load(bhc)

In [0]:
try:
    #Call Parent Tests - with field exclusion comma separated list
    if len(fields_to_exclude) !=  0:
        str_fields_to_exclude = ",".join(fields_to_exclude)
    else:
        str_fields_to_exclude = ""
    
    child_notebook_to_run = "PaymentPending_TESTS"
    pp_results = dbutils.notebook.run(child_notebook_to_run, timeout_seconds=1800, arguments={
        "fields_to_exclude": str_fields_to_exclude,
        "state_under_test": state_under_test
    })

    from dataclasses import dataclass
    @dataclass
    class TestResult:
        test_field: str =""
        status: str =""
        message: str = ""
        test_from_state: str=""
        test_name: str=""    

    #Convert string of results back into list of TestResults class objects
    all_test_results = [TestResult(**d) for d in json.loads(pp_results)]
    print (all_test_results)

except Exception as e:
    message =  f"Notebook Execution Failed for: {child_notebook_to_run} -- execution failed with Message : {str(e)}\nFurther Execution Stopped"    
    dbutils.notebook.exit(message)    

In [0]:
#Used to Quickly reload the function file containing the test, rather than restart python
import importlib
importlib.reload(as_tests)

In [0]:
#DUMMY TEST CASEs
if "fakeappealsubmitted_test1" not in fields_to_exclude:
    all_test_results.append(as_tests.testcase1())
    print(all_test_results)

if "fakeappealsubmitted_test2" not in fields_to_exclude:
    all_test_results.append(as_tests.testcase2())
    print(all_test_results)



In [0]:
#######################
#PROCESS RESULTS
#######################
from collections import defaultdict
from datetime import datetime

#Convert Results Classes to string
all_test_results_string = json.dumps([asdict(r) for r in all_test_results])
lines = []

#Print Results
grouped = defaultdict(list)
for r in all_test_results:
        grouped[r.test_from_state].append(r)
total_tests = len(all_test_results)
total_passed = sum(1 for r in all_test_results if r.status == "PASS")
total_failed = sum(1 for r in all_test_results if r.status == "FAIL")

lines.append("TEST EXECUTION REPORT")
lines.append(f"Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}")
lines.append("-" * 80)
lines.append(f"Total tests : {total_tests}")
lines.append(f"Passed      : {total_passed}")
lines.append(f"Failed      : {total_failed}")
lines.append("-" * 80)

for state in sorted(grouped):  
    state_results = grouped[state]
    state_passed = sum(1 for r in state_results if r.status == "PASS")
    state_failed = sum(1 for r in state_results if r.status == "FAIL")
    lines.append(f"\nSTATE: {state}")
    lines.append(f"Tests: {len(state_results)} | Passed: {state_passed} | Failed: {state_failed}")
    lines.append("-" * 60)

print("\n".join(lines))

#Check If if this notebook is parent notebook(so produce report), or has been called by another(return data to parent)
if state_under_test == this_notebook_state:
    #Call Print Test Report Notebook
    dbutils.notebook.run( base_path + "Shared_Notebooks/Common_Produce_TestResults", 600, {
    "all_test_results_string":all_test_results_string,
    "state_under_test":state_under_test,
    "base_path":base_path,
    "test_results_path": test_results_path,
    "run_user":run_user,
    "run_tag":run_tag,
    "run_by_automation_name": run_by_automation_name,
    "run_start_datetime": run_start_datetime.strftime("%Y-%m-%d %H:%M:%S"), 
    })
    
#else return to parent notebook
else:
    print(f"Exiting Notebook : {this_notebook_state} and returning Data to Parent notebook : {state_under_test}")
    dbutils.notebook.exit(all_test_results_string)